In [2]:
from garminconnect import Garmin
import fitdecode
import zipfile
import io

In [3]:
client = Garmin("ju.roulle@gmail.com", "Julien35830")
client.login()
activities = client.get_activities(0, 1)

In [4]:
activity = activities[0]
activity_id = activity["activityId"]

In [6]:
data = client.download_activity(activity_id, dl_fmt=client.ActivityDownloadFormat.ORIGINAL)

with zipfile.ZipFile(io.BytesIO(data), "r") as zip_file:

    for file_name in zip_file.namelist():
        file_content = zip_file.read(file_name)
        
        # Extract all files in the zip archive into the memory file
        memory_file = io.BytesIO()

        # Read the contents of the extracted file into an io.BytesIO buffer
        memory_file.write(file_content)

        # Reset the memory file's position to the beginning
        memory_file.seek(0)
        with fitdecode.FitReader(memory_file) as fit_file:
            for frame in fit_file:
                if frame.frame_type == fitdecode.FIT_FRAME_DATA:
                    if frame.name == 'record':
                        break